In [2]:
import sys
import requests #package for http requests
import bs4 # package for html parsing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from datetime import date
import seaborn as sns

pd.set_option('display.max_rows', 5000)

In [5]:
total_data = pd.read_csv('C:\\Users\\ashto\\Desktop\\INFO Raw Data2021.csv')
total_data['Driver'] = total_data['Driver'].str.rsplit(' ',1).str[0]
total_data['Driver'] = total_data['Driver'].str.replace('  ', ' ')
total_data['Driver'] = total_data['Driver'].str.replace('   ', ' ')
total_data['Driver'] = total_data['Driver'].str.replace('RÃ¤ikkÃ¶nen','Raikkonen')

In [6]:
total_data = total_data.rename(columns={'Pos_x':'Ending Position','Pos_y':'Starting Position'})
total_data = total_data[['Driver','Car','Laps','Time/Retired','PTS','Starting Position','Ending Position','Year']]
total_data.loc[total_data['Starting Position'] == 'NC', 'Starting Position'] = 0
total_data.loc[total_data['Ending Position'] == 'DQ', 'Ending Position'] = 0
total_data.loc[total_data['Ending Position'] == 'NC', 'Ending Position'] = 0


total_data['Starting Position'] = total_data['Starting Position'].fillna(0)
total_data.head(100)

,Driver,Car,Laps,Time/Retired,PTS,Starting Position,Ending Position,Year
0,Alain Prost,Williams Renault,72.0,1:38:45.082,10.0,1.0,1,1993
1,Ayrton Senna,McLaren Ford,72.0,+79.824s,6.0,2.0,2,1993
2,Mark Blundell,Ligier Renault,71.0,+1 lap,4.0,8.0,3,1993
3,Christian Fittipaldi,Minardi Ford,71.0,+1 lap,3.0,13.0,4,1993
4,Jyrki Jarvilehto,Sauber,70.0,+2 laps,2.0,6.0,5,1993
5,Gerhard Berger,Ferrari,69.0,DNF,1.0,15.0,6,1993
6,Derek Warwick,Footwork Mugen Honda,69.0,DNF,0.0,22.0,7,1993
7,Martin Brundle,Ligier Renault,57.0,DNF,0.0,12.0,0,1993
8,Michele Alboreto,Lola Ferrari,55.0,DNF,0.0,25.0,0,1993
9,Erik Comas,Larrousse Lamborghini,51.0,DNF,0.0,19.0,0,1993


In [7]:
total_data['Positions Gained'] = total_data['Starting Position'].astype(int) - total_data['Ending Position'].astype(int)
total_data.head()

,Driver,Car,Laps,Time/Retired,PTS,Starting Position,Ending Position,Year,Positions Gained
0,Alain Prost,Williams Renault,72.0,1:38:45.082,10.0,1.0,1,1993,0
1,Ayrton Senna,McLaren Ford,72.0,+79.824s,6.0,2.0,2,1993,0
2,Mark Blundell,Ligier Renault,71.0,+1 lap,4.0,8.0,3,1993,5
3,Christian Fittipaldi,Minardi Ford,71.0,+1 lap,3.0,13.0,4,1993,9
4,Jyrki Jarvilehto,Sauber,70.0,+2 laps,2.0,6.0,5,1993,1


In [8]:
total_data.to_csv('C:\\Users\\ashto\\Desktop\\INFO Cleaned FINAL Data.csv')

In [9]:
salary_url = 'https://drive.google.com/file/d/1AG_ReOXKGeYSe2WhPk1Q2_G96wGR1GdT/view?usp=sharing'
salary_url2='https://drive.google.com/uc?id=' + salary_url.split('/')[-2]
salary = pd.read_csv(salary_url2,encoding='windows-1252')
salary.head()

,Driver,Salary,Year
0,Kimi Raikkonen,7525000,2021
1,Antonio Giovinazzi,684000,2021
2,Fernando Alonso,20500000,2021
3,Esteban Ocon,4652000,2021
4,Sebastian Vettel,17000000,2021


In [10]:
merge = pd.merge(total_data,salary,on=['Driver','Year'],how='left')
merge.head()
salary_results = merge = pd.merge(total_data,salary,on=['Driver','Year'])
salary_results.head()
len(salary_results)

3746

In [11]:
salary_results.to_csv('C:\\Users\\ashto\\Desktop\\INFO FINAL Salary Merged Data.csv')

In [12]:
salary_results['Cost per Point'] = salary_results['Salary']/salary_results['PTS']
salary_results.head()

,Driver,Car,Laps,Time/Retired,PTS,Starting Position,Ending Position,Year,Positions Gained,Salary,Cost per Point
0,Sebastian Vettel,Red Bull Racing Renault,58.0,1:29:30.259,25.0,1.0,1,2011,0,9280000,371200.000000
1,Sebastian Vettel,Red Bull Racing Renault,56.0,1:37:39.832,25.0,1.0,1,2011,0,9280000,371200.000000
2,Sebastian Vettel,Red Bull Racing Renault,56.0,+5.198s,18.0,1.0,2,2011,-1,9280000,515555.555556
3,Sebastian Vettel,Red Bull Racing Renault,58.0,1:30:17.558,25.0,1.0,1,2011,0,9280000,371200.000000
4,Sebastian Vettel,Red Bull Racing Renault,66.0,1:39:03.301,25.0,2.0,1,2011,1,9280000,371200.000000


In [13]:
#drop players with zero points from Cost per Point analysis
salary_cleaned = salary_results[salary_results['Cost per Point']!=np.inf]
salary_cleaned.to_csv('C:\\Users\\ashto\\Desktop\\INFO Phase III Data.csv')
#salary_cleaned = salary_cleaned.dropna()
salary_cleaned.head()

,Driver,Car,Laps,Time/Retired,PTS,Starting Position,Ending Position,Year,Positions Gained,Salary,Cost per Point
0,Sebastian Vettel,Red Bull Racing Renault,58.0,1:29:30.259,25.0,1.0,1,2011,0,9280000,371200.000000
1,Sebastian Vettel,Red Bull Racing Renault,56.0,1:37:39.832,25.0,1.0,1,2011,0,9280000,371200.000000
2,Sebastian Vettel,Red Bull Racing Renault,56.0,+5.198s,18.0,1.0,2,2011,-1,9280000,515555.555556
3,Sebastian Vettel,Red Bull Racing Renault,58.0,1:30:17.558,25.0,1.0,1,2011,0,9280000,371200.000000
4,Sebastian Vettel,Red Bull Racing Renault,66.0,1:39:03.301,25.0,2.0,1,2011,1,9280000,371200.000000


In [14]:
len(salary_cleaned)

1926